In [ ]:
import batoid
import os
import yaml
import numpy as np
import time
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
LSST_r_fn = os.path.join(batoid.datadir, "LSST", "LSST_r2.yaml")
config = yaml.load(open(LSST_r_fn))
fiducial_telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
theta_x = 0
theta_y = 0
wavelength = 620e-9

telescope = fiducial_telescope.withGloballyShiftedOptic('LSST.LSSTCamera', batoid.Vec3(0,0,1e-3))

telescope.dist = 20.0
telescope.pupil_size = 10.0
telescope.sphereRadius = 10.5
telescope.inMedium = batoid.Air()

nx=1536
scale, fftpsf = batoid.fftPSF(telescope, wavelength, theta_x, theta_y, nx=nx, pad_factor=1)

extent = scale*fftpsf.shape[0]/2*np.r_[-1., 1., -1., 1.] # radians
extent -= scale/2
extent *= 206265 # arcsec
extent /= 0.2 # pixels
extent *= 10 # microns
plt.imshow(fftpsf/fftpsf.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF")
plt.show()

fftpsf_sub = fftpsf[16::32, 16::32]
plt.imshow(fftpsf_sub/fftpsf_sub.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF downsampled")
plt.show()

In [ ]:
theta_x = 0
theta_y = 1.75
wavelength = 620e-9

telescope = fiducial_telescope.withGloballyShiftedOptic('LSST.LSSTCamera', batoid.Vec3(0,0,1e-3))

telescope.dist = 20.0
telescope.pupil_size = 10.0
telescope.sphereRadius = 10.5
telescope.inMedium = batoid.Air()

nx=1536
scale, fftpsf = batoid.fftPSF(telescope, wavelength, theta_x, theta_y, nx=nx, pad_factor=1)

extent = scale*fftpsf.shape[0]/2*np.r_[-1., 1., -1., 1.] # radians
extent -= scale/2
extent *= 206265 # arcsec
extent /= 0.2 # pixels
extent *= 10 # microns
plt.imshow(fftpsf/fftpsf.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF")
plt.show()

fftpsf_sub = fftpsf[16::32, 16::32]
plt.imshow(fftpsf_sub/fftpsf_sub.sum(), extent=extent)
plt.colorbar()
plt.title("FFT PSF downsampled")
plt.show()

In [ ]:
t0 = time.time()

wavelength = 500e-9
theta_x = 0
theta_y = 1.75

xcos = np.sin(theta_x*np.pi/180)
ycos = np.sin(theta_y*np.pi/180)
zcos = -np.sqrt(1.0 - xcos**2 - ycos**2)
orig_rays = batoid.circularGrid(18.0, 4.5, 2.0, xcos, ycos, zcos,
                                600, 6000, wavelength, batoid.Air())

traced_rays, _ = telescope.trace(orig_rays)    
goodRays = batoid.trimVignetted(traced_rays)
xmean = np.mean(goodRays.x)
ymean = np.mean(goodRays.y)
xs = np.linspace(xmean-500e-6, xmean+500e-6, 32)
ys = np.linspace(ymean-500e-6, ymean+500e-6, 32)
xs, ys = np.meshgrid(xs, ys)

huygensPsf = batoid.huygensPSF(telescope, xs=xs, ys=ys, zs=None, rays=orig_rays, saveRays=False)

plt.imshow(huygensPsf/huygensPsf.sum())
plt.colorbar()
plt.title("Huygens PSF")
plt.show()

t1 = time.time()
print("Huygens PSF took {:6.1f} seconds to compute".format(t1-t0))